# mastercan to datafreame to compare|

In [169]:
lx1_masterscan_path = r'test_resources\small_test\small_test.sc'
lx2_masterscan_path = r'test_resources\small_test\small_test-lx2.sc'

In [170]:
from lx.spectraTools import loadSC

In [171]:
lx1_masterscan = loadSC(lx1_masterscan_path)
lx2_masterscan = loadSC(lx2_masterscan_path)

Loading SC test_resources\small_test\small_test.sc ...
Loading SC test_resources\small_test\small_test-lx2.sc ...


# compare ms1

In [172]:
lx1_ms1 = [(idx, se.precurmass) for idx, se in enumerate(lx1_masterscan.listSurveyEntry)]
lx2_ms1 = [(idx, se.precurmass) for idx, se in enumerate(lx2_masterscan.listSurveyEntry)]

In [173]:
import pandas as pd

In [174]:
lx1_df = pd.DataFrame(lx1_ms1, columns=['idx_lx1','precurmass_lx1'])
lx2_df = pd.DataFrame(lx2_ms1, columns=['idx_lx2', 'precurmass_lx2'])

In [175]:
def find_common_elements(lx1_df, lx2_df, tolerance = 0.05):

    lx1_df.sort_values(by=['precurmass_lx1'], inplace=True)
    lx2_df.sort_values(by=['precurmass_lx2'], inplace=True)
    merged = pd.merge_asof(lx1_df, lx2_df, left_on='precurmass_lx1', right_on ='precurmass_lx2' , tolerance=tolerance,direction='nearest')
    merged2 = pd.merge_asof(lx2_df, lx1_df, left_on='precurmass_lx2', right_on = 'precurmass_lx1', tolerance=tolerance,direction='nearest')

    return pd.concat([merged, merged2], sort=False)


In [176]:
common = find_common_elements(lx1_df, lx2_df)


In [177]:
common

,idx_lx1,precurmass_lx1,idx_lx2,precurmass_lx2
0,0.0,360.180357,284.0,360.226913
1,1.0,360.231992,284.0,360.226913
2,2.0,360.253138,284.0,360.226913
3,3.0,360.310635,283.0,360.310385
4,4.0,360.323470,283.0,360.310385
...,...,...,...,...
280,1841.0,986.817741,4.0,986.811011
281,1842.0,986.926874,3.0,986.920202
282,1843.0,987.930105,2.0,987.923407
283,NaN,NaN,1.0,988.781073


# ms2 in matching ms1s

In [178]:
common = common.dropna().drop_duplicates(subset=['precurmass_lx1', 'precurmass_lx2'])

In [179]:
common

,idx_lx1,precurmass_lx1,idx_lx2,precurmass_lx2
0,0.0,360.180357,284.0,360.226913
1,1.0,360.231992,284.0,360.226913
2,2.0,360.253138,284.0,360.226913
3,3.0,360.310635,283.0,360.310385
4,4.0,360.323470,283.0,360.310385
...,...,...,...,...
21,133.0,394.291622,263.0,394.279623
22,133.0,394.291622,262.0,394.287359
178,1446.0,812.616611,106.0,812.637732
186,1458.0,815.635841,98.0,815.656775


In [305]:
def ms2_common(common):
    for row in common.itertuples():
        listMSMS_lx1 = lx1_masterscan.listSurveyEntry[int(row.idx_lx1)].listMSMS
        listMSMS_lx2 = lx2_masterscan.listSurveyEntry[int(row.idx_lx2)].listMSMS
        s1 = pd.Series((x.mass for x in listMSMS_lx1), name='ms2_lx1').sort_values()
        s2 = pd.Series((x.mass for x in listMSMS_lx2), name='ms2_lx2').sort_values()
        m1 = pd.merge_asof(s1, s2, left_on= 'ms2_lx1', right_on='ms2_lx2', direction='nearest', tolerance=0.05)
        m2 = pd.merge_asof(s2, s1, left_on= 'ms2_lx2', right_on='ms2_lx1', direction='nearest', tolerance=0.05)
        c = pd.concat([m1, m2], sort=False)
        c.drop_duplicates(inplace=True)
        c['common_idx'] = row.Index
        yield c

def ms2_uncommon(common): 
    for c in ms2_common(common):
        yield(c[c.isna().any(axis = 1)])

In [306]:
uncommon = pd.concat(ms2_uncommon(common))

In [240]:
# handrolled find nearest closest, but with tolerance
# https://stackoverflow.com/questions/57361453/matching-two-lists-containing-slightly-differing-float-values-by-allowing-a-tole?

In [307]:
#mismatched ms2s
uncommon

,ms2_lx1,ms2_lx2,common_idx
59,360.100886,NaN,0
60,360.112812,NaN,0
66,360.251131,NaN,0
73,368.628960,NaN,0
74,382.975252,NaN,0
...,...,...,...
57,815.726919,NaN,186
58,815.818858,NaN,186
59,843.639458,NaN,186
60,847.201753,NaN,186


In [309]:
# the most mismatched ms2s
uncommon.common_idx.value_counts()

365     276
364     276
436     208
435     208
1837    177
       ... 
219       1
1725      1
294       1
296       1
1709      1
Name: common_idx, Length: 361, dtype: int64

In [311]:
#precursor infor for mismatched ms2a
common.loc[uncommon.common_idx.value_counts().head(10).index]

,idx_lx1,precurmass_lx1,idx_lx2,precurmass_lx2
365,365.0,462.963002,240.0,462.950064
364,364.0,462.950504,240.0,462.950064
436,436.0,484.945102,236.0,484.932165
435,435.0,484.932591,236.0,484.932165
1837,1837.0,981.279058,8.0,981.265722
851,851.0,607.959030,197.0,607.917059
1834,1834.0,980.278584,11.0,980.271887
186,186.0,411.222485,259.0,411.233999
186,1458.0,815.635841,98.0,815.656775
1729,1729.0,906.838951,48.0,906.837805


In [324]:
#most unmatched ms2s
uncommon.loc[uncommon.common_idx.isin(uncommon.common_idx.value_counts().head(10).index)]